In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http:

model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="final_dataset.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2692 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,

        # changed epochs to 1 for a full run

        num_train_epochs = 1,
        max_steps = -1,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/2692 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
7.135 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,692 | Num Epochs = 1 | Total steps = 337
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.596500
2,2.623100
3,2.769500
4,2.543000
5,2.328300
6,2.039300
7,1.553300
8,1.188000
9,0.911200
10,0.859900


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

523.8478 seconds used for training.
8.73 minutes used for training.
Peak reserved memory = 7.135 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 18.037 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
# 1. Make sure your prompt template is defined (as you said, copy from above)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# 2. Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# 3. Define your test prompt
# MUST match the format you trained on
instruction = "You are an expert interview question generator. Generate an interview question based on the parameters provided in the input."
input_params = "Domain: Behavioral\nDifficulty: Medium" # Try any new domain!

# 4. Tokenize the prompt
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction,    # Your specific instruction
        input_params,   # Your new test input
        "",             # Output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# 5. Generate the response
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True) # Increased tokens
response_text = tokenizer.batch_decode(outputs)[0]

# 6. Print the result
print(response_text)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert interview question generator. Generate an interview question based on the parameters provided in the input.

### Input:
Domain: Behavioral
Difficulty: Medium

### Response:
Describe a scenario where you had to adapt your data analysis approach due to unexpected changes in data quality or availability. What were the challenges you faced, and how did you overcome them?<|end_of_text|>


### Performance Benchmarking (Speed)

In [ ]:
import time

# --- 1. Set up your test prompt ---
# (Using the same prompt format as before)
instruction = "You are an expert interview question generator. Generate an interview question based on the parameters provided in the input."
input_params = "Domain: Machine Learning\nDifficulty: Advanced"

inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction,
        input_params,
        "",             # Output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# --- 2. WARMUP RUN ---
# We run this once to "warm up" the GPU and avoid timing the model load.
print("Running warmup...")
_ = model.generate(**inputs, max_new_tokens = 10, use_cache = True)
print("Warmup complete. Starting benchmark...")
print("---------------------------------")


# --- 3. BENCHMARK ---
max_new_tokens_to_gen = 128
start_time = time.perf_counter() # Start the timer

# Generate the text
outputs = model.generate(**inputs, max_new_tokens = max_new_tokens_to_gen, use_cache = True)

end_time = time.perf_counter() # Stop the timer

# Decode the output (we need to count the new tokens)
response_text = tokenizer.batch_decode(outputs)[0]
new_tokens = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0] # Get only the *new* tokens
num_new_tokens = len(tokenizer.encode(new_tokens))

# --- 4. CALCULATE AND PRINT RESULTS ---
total_time = end_time - start_time
tokens_per_second = num_new_tokens / total_time

print(f"--- Inference Performance ---")
print(f"Total time for generation: {total_time:.2f} seconds")
print(f"Number of new tokens generated: {num_new_tokens}")
print(f"Tokens per second (Throughput): {tokens_per_second:.2f} T/s")
print("---------------------------------")
print(f"\nModel Output:\n{response_text}")

Running warmup...
Warmup complete. Starting benchmark...
---------------------------------
--- Inference Performance ---
Total time for generation: 2.76 seconds
Number of new tokens generated: 43
Tokens per second (Throughput): 15.61 T/s
---------------------------------

Model Output:
<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert interview question generator. Generate an interview question based on the parameters provided in the input.

### Input:
Domain: Machine Learning
Difficulty: Advanced

### Response:
Describe the core differences between supervised and unsupervised machine learning, and explain a scenario where each approach would be most suitable. Additionally, discuss the potential challenges and considerations when implementing unsupervised learning techniques.<|end_of_text|>


### Quality Benchmarking (Accuracy)

In [ ]:
# --- 1. Define your "benchmark" prompts ---
benchmark_prompts = [
    {"domain": "Python", "difficulty": "Basic"},
    {"domain": "Python", "difficulty": "Medium"},
    {"domain": "Python", "difficulty": "Advanced"},
    {"domain": "SQL", "difficulty": "Medium"},
    {"domain": "SQL", "difficulty": "Advanced"},
    {"domain": "Machine Learning", "difficulty": "Medium"},
    {"domain": "Deep Learning", "difficulty": "Advanced"},
    {"domain": "System Design", "difficulty": "Advanced"},
]

print("--- Running Quality Benchmark ---")
print("="*30)

# Make sure model is in inference mode
FastLanguageModel.for_inference(model)

for item in benchmark_prompts:
    # --- 2. Format the prompt ---
    instruction = "You are an expert interview question generator. Generate an interview question based on the parameters provided in the input."
    input_params = f"Domain: {item['domain']}\nDifficulty: {item['difficulty']}"

    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction,
            input_params,
            "",
        )
    ], return_tensors = "pt").to("cuda")

    # --- 3. Generate the response ---
    outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True,
                             do_sample=True, temperature=0.7) # Use sampling for variety

    response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]

    # --- 4. Print a clean report ---
    print(f"PROMPT: Domain={item['domain']}, Difficulty={item['difficulty']}")
    print(f"GENERATED QUESTION:\n{response_text}\n")
    print("="*30)

--- Running Quality Benchmark ---
PROMPT: Domain=Python, Difficulty=Basic
GENERATED QUESTION:
What is a Django model?<|end_of_text|>

PROMPT: Domain=Python, Difficulty=Medium
GENERATED QUESTION:
Explain the concept of a tree data structure and describe a scenario where using a tree would be more efficient than using a linked list.<|end_of_text|>

PROMPT: Domain=Python, Difficulty=Advanced
GENERATED QUESTION:
Describe the key differences between a Python class and a Python instance, and explain how you would design a class hierarchy to model a complex real-world system.<|end_of_text|>

PROMPT: Domain=SQL, Difficulty=Medium
GENERATED QUESTION:
Given an 'Employees' table with 'EmployeeID' and 'ManagerID' columns, how would you identify employees who have no managers? Explain your approach and any assumptions you make about the data.<|end_of_text|>

PROMPT: Domain=SQL, Difficulty=Advanced
GENERATED QUESTION:
When would you use a materialized view versus a regular view in PostgreSQL, and wh

<a name="Save"></a>
### Saving, loading finetuned models


In [ ]:
# --- Save the 16-bit (full quality) model ---
# 'merged_16bit'
print("Starting to save 16-bit merged model... (This will be slow)")
model.save_pretrained_merged(
    "merged_16bit",
    tokenizer,
    save_method = "merged_16bit",
)
print("Done saving 16-bit model.")

Starting to save 16-bit merged model... (This will be slow)


config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [01:43<00:00, 25.85s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:55<00:00, 13.90s/it]


Unsloth: Merge process complete. Saved to `/content/merged_16bit`
Done saving 16-bit model.


In [ ]:
print("\nZipping the 16-bit model folder... (This may take several minutes)")
!zip -r merged_16bit.zip ./merged_16bit
print("Done zipping 16-bit model.")


Zipping the 16-bit model folder... (This may take several minutes)
  adding: merged_16bit/ (stored 0%)
  adding: merged_16bit/.cache/ (stored 0%)
  adding: merged_16bit/.cache/huggingface/ (stored 0%)
  adding: merged_16bit/.cache/huggingface/.gitignore (stored 0%)
  adding: merged_16bit/.cache/huggingface/download/ (stored 0%)
  adding: merged_16bit/.cache/huggingface/download/model.safetensors.index.json.lock (stored 0%)
  adding: merged_16bit/.cache/huggingface/download/model.safetensors.index.json.metadata (deflated 26%)
  adding: merged_16bit/.cache/huggingface/download/model-00001-of-00004.safetensors.lock (stored 0%)
  adding: merged_16bit/.cache/huggingface/download/model-00002-of-00004.safetensors.lock (stored 0%)
  adding: merged_16bit/.cache/huggingface/download/model-00001-of-00004.safetensors.metadata (deflated 30%)
  adding: merged_16bit/.cache/huggingface/download/model-00004-of-00004.safetensors.lock (stored 0%)
  adding: merged_16bit/.cache/huggingface/download/model-

In [ ]:
# --- Save the 4-bit (quantized) model ---
# 'merged_4bit'
print("\nStarting to save 4-bit merged model... (This is fast)")
model.save_pretrained_merged(
    "merged_4bit",
    tokenizer,
    save_method = "merged_4bit_forced",
)
print("Done saving 4-bit model.")


Starting to save 4-bit merged model... (This is fast)
Unsloth: Merging LoRA weights into 4bit model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Unsloth: Merging finished.
Unsloth: Found skipped modules: ['model.layers.1.mlp.gate_proj', 'model.layers.1.mlp.up_proj', 'model.layers.1.mlp.down_proj', 'lm_head']. Updating config.
Unsloth: Saving merged 4bit model to merged_4bit...
Unsloth: Merged 4bit model saved.
Unsloth: Merged 4bit model process completed.
Done saving 4-bit model.


In [ ]:
print("Zipping the 4-bit model folder...")
!zip -r merged_4bit.zip ./merged_4bit
print("Done zipping 4-bit model.")

Zipping the 4-bit model folder...
  adding: merged_4bit/ (stored 0%)
  adding: merged_4bit/special_tokens_map.json (deflated 71%)
  adding: merged_4bit/model-00001-of-00002.safetensors (deflated 8%)
  adding: merged_4bit/generation_config.json (deflated 36%)
  adding: merged_4bit/model-00002-of-00002.safetensors (deflated 21%)
  adding: merged_4bit/tokenizer.json (deflated 85%)
  adding: merged_4bit/model.safetensors.index.json (deflated 96%)
  adding: merged_4bit/tokenizer_config.json (deflated 96%)
  adding: merged_4bit/config.json (deflated 57%)
Done zipping 4-bit model.


In [ ]:
# This saves the tiny adapter files (like adapter_model.safetensors)
# to a folder named 'lora_model'
print("Saving LoRA adapters (this is fast)...")
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
print("Done saving adapters.")

Saving LoRA adapters (this is fast)...
Done saving adapters.


In [ ]:
print("Zipping the 'lora_model' folder...")
!zip -r lora_model.zip ./lora_model
print("Done zipping adapters.")

Zipping the 'lora_model' folder...
  adding: lora_model/ (stored 0%)
  adding: lora_model/adapter_config.json (deflated 57%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)
  adding: lora_model/special_tokens_map.json (deflated 71%)
  adding: lora_model/tokenizer.json (deflated 85%)
  adding: lora_model/README.md (deflated 65%)
  adding: lora_model/tokenizer_config.json (deflated 96%)
Done zipping adapters.


In [ ]:
from google.colab import files

# Download the 4-bit model first (recommended)
print("Preparing 4-bit model for download. Please wait...")
files.download('merged_4bit.zip')

Preparing 4-bit model for download. Please wait...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

print("Preparing 'lora_model.zip' for download...")
files.download('lora_model.zip')

Preparing 'lora_model.zip' for download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>